## Dissertation Dataset Exploration
### Gavin Rolls

Now that I've queried and cleaned my data as well as run a few dummy models, I'm going to try to do some data exploration to understand the underlying dataset a little bit better

### Imports and Clean Up

In [51]:
# Library Imports

#Basics
import pandas as pd
import geopandas as gpd

#Shapely
from shapely.geometry import Polygon, MultiPolygon

# Machine Learning
from sklearn.model_selection import train_test_split

#ML from mljar-supervised
from supervised.automl import AutoML

In [9]:
# Import Cleaned Employment Geography File

empl_geog = gpd.read_file('data/combined_data/empl_geog.geojson')

empl_geog.head()

,LSOA11CD,LSOA11NM_x,population,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,...,LSOA11NM_y,total,num_overture_buildings,num_osm_buildings,num_places,overture_building_poly,osm_building_poly,place_points,category_list,geometry
0,E01000001,City of London 001A,1465,lsoa2011:E01000001 : City of London 001A,0,0,0,0,10,0,...,City of London 001A,14360,29,86,243,"MULTIPOLYGON (((-0.0968818 51.5148582, -0.0969...","MULTIPOLYGON (((-0.0983297 51.5158158, -0.0983...","MULTIPOINT ((-0.0987982 51.5176724), (-0.09860...","[pub, coffee_shop, immigration_law, shared_off...","POLYGON ((-0.09729 51.52158, -0.09652 51.52027..."
1,E01000002,City of London 001B,1436,lsoa2011:E01000002 : City of London 001B,0,0,0,0,0,0,...,City of London 001B,45605,40,96,419,"MULTIPOLYGON (((-0.0932921 51.5152927, -0.0932...","MULTIPOLYGON (((-0.0945314 51.5147658, -0.0944...","MULTIPOINT ((-0.0958288 51.5168656), (-0.09565...","[business_manufacturing_and_supply, education,...","POLYGON ((-0.08813 51.51941, -0.08929 51.51752..."
2,E01000003,City of London 001C,1346,lsoa2011:E01000003 : City of London 001C,0,0,0,0,0,0,...,City of London 001C,760,10,29,55,"MULTIPOLYGON (((-0.0959197 51.5205155, -0.0961...","MULTIPOLYGON (((-0.0959637 51.5209148, -0.0959...","MULTIPOINT ((-0.0976057 51.5226912), (-0.09757...","[health_and_medical, hospital, parking, indust...","POLYGON ((-0.09679 51.52325, -0.09647 51.52282..."
3,E01000005,City of London 001E,985,lsoa2011:E01000005 : City of London 001E,0,10,0,0,0,0,...,City of London 001E,22735,39,133,534,"MULTIPOLYGON (((-0.07593 51.5108457, -0.076125...","MULTIPOLYGON (((-0.0757822 51.511071, -0.07577...","MULTIPOINT ((-0.0799241 51.5156172), (-0.07992...","[professional_services, dance_club, insurance_...","POLYGON ((-0.07323 51.51000, -0.07553 51.50974..."
4,E01000006,Barking and Dagenham 016A,1703,lsoa2011:E01000006 : Barking and Dagenham 016A,0,0,0,0,0,0,...,Barking and Dagenham 016A,105,0,280,11,None,"MULTIPOLYGON (((0.085895 51.5386732, 0.0859413...","MULTIPOINT ((0.0856198 51.538706), (0.0858424 ...","[landmark_and_historical_building, constructio...","POLYGON ((0.09115 51.53909, 0.09326 51.53787, ..."


In [59]:
# Import Category Encoded File

encoded_categories = pd.read_csv('data/combined_data/empl_geog_category_encoded.csv')

#Drop extraneous column
encoded_categories = encoded_categories.drop(columns=['Unnamed: 0'])

#Store category columns
category_columns = encoded_categories.columns[1:]

encoded_categories.head()

,LSOA11CD,category_list_3d_printing_service,category_list_abortion_clinic,category_list_abuse_and_addiction_treatment,category_list_accommodation,category_list_accountant,category_list_acoustical_consultant,category_list_active_life,category_list_acupuncture,category_list_addiction_rehabilitation_center,...,category_list_women's_clothing_store,category_list_women's_health_clinic,category_list_wood_and_pulp,category_list_woodworking_supply_store,category_list_writing_service,category_list_yoga_instructor,category_list_yoga_studio,category_list_youth_organizations,category_list_zoo,category_list_None
0,E01000001,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,E01000002,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,E01000003,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E01000005,0.0,0.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,E01000006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# Geometric Features Extraction (copied from earlier)

def extract_multipolygon_features(multipolygon, lsoa_geometry):
    if isinstance(multipolygon, MultiPolygon):
        polygons = list(multipolygon.geoms)
    elif isinstance(multipolygon, Polygon):
        polygons = [multipolygon]
    else:
        return pd.Series({
            'num_polygons': 0,
            'total_area': 0,
            'total_perimeter': 0,
            'area_ratio': 0,
            'avg_area': 0,
            'avg_perimeter': 0
        })

    num_polygons = len(polygons)
    areas = [polygon.area for polygon in polygons]
    perimeters = [polygon.length for polygon in polygons]
    total_area = sum(areas)
    total_perimeter = sum(perimeters)
    avg_area = total_area / num_polygons if num_polygons > 0 else 0
    avg_perimeter = total_perimeter / num_polygons if num_polygons > 0 else 0

    #Built-up area ratio could result in a div by zero error if there are no buildings in an LSOA so this logic has to be included here
    try:
        area_ratio = total_area / lsoa_geometry.area
    except ZeroDivisionError:
        area_ratio = 0

    return pd.Series({
        'num_polygons': num_polygons,
        'total_area': total_area,
        'area_ratio': area_ratio,
        'total_perimeter': total_perimeter,
        'avg_area': avg_area,
        'avg_perimeter': avg_perimeter
    })


In [53]:
#Apply function above

#Both Building Set Geometry Features
osm_geometry_features = empl_geog.apply(
    lambda row: extract_multipolygon_features(row['osm_building_poly'], row['geometry']),
    axis=1
)

empl_geog_osm_features = pd.concat([empl_geog, osm_geometry_features], axis=1)


In [55]:
# Combine Datasets (Same Logic as in EarlyModels.ipynb)

all_data = pd.merge(empl_geog_osm_features, encoded_categories, on='LSOA11CD')

all_data.head()

,LSOA11CD,LSOA11NM_x,population,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,...,category_list_women's_clothing_store,category_list_women's_health_clinic,category_list_wood_and_pulp,category_list_woodworking_supply_store,category_list_writing_service,category_list_yoga_instructor,category_list_yoga_studio,category_list_youth_organizations,category_list_zoo,category_list_None
0,E01000001,City of London 001A,1465,lsoa2011:E01000001 : City of London 001A,0,0,0,0,10,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,E01000002,City of London 001B,1436,lsoa2011:E01000002 : City of London 001B,0,0,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,E01000003,City of London 001C,1346,lsoa2011:E01000003 : City of London 001C,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E01000005,City of London 001E,985,lsoa2011:E01000005 : City of London 001E,0,10,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,E01000006,Barking and Dagenham 016A,1703,lsoa2011:E01000006 : Barking and Dagenham 016A,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Data Exploration and Analysis

In [71]:
#AutoML Test

# create training and testing data
features = ['num_polygons', 'total_area', 'total_perimeter', 'area_ratio', 'avg_area', 'avg_perimeter', 'num_osm_buildings', 'num_places', 'population'] + list(category_columns)
target = 'total'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data[features], all_data[target], test_size=0.2, random_state=42)

# the results_path should point to a folder that you have already created to store the model results
automl = AutoML(results_path="automl_results/first_model/", mode='Explain')
# fit the model
automl.fit(X_train, y_train)

Linear algorithm was disabled.
AutoML directory: automl_results/first_model/
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 5958.176481 trained in 0.61 seconds


/Users/gavinrolls/anaconda3/envs/urbsim/lib/python3.11/site-packages/supervised/utils/shap.py:38: UserWarning: Disable SHAP explanations because of number of columns > 500.


2_DecisionTree rmse 12594.497399 trained in 81.02 seconds
* Step default_algorithms will try to check up to 3 models


/Users/gavinrolls/anaconda3/envs/urbsim/lib/python3.11/site-packages/supervised/preprocessing/scale.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[ 896.71862793]
 [ 189.04449463]
 [ 240.83198547]
 ...
 [ 126.07458496]
 [ 208.0974884 ]
 [2291.09619141]]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
/Users/gavinrolls/anaconda3/envs/urbsim/lib/python3.11/site-packages/supervised/preprocessing/scale.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[4.79517609e+02]
 [3.79830475e+02]
 [1.71133194e+02]
 [6.64738731e+01]
 [1.98407059e+02]
 [1.85534561e+02]
 [2.08337143e+02]
 [4.54004333e+02]
 [5.81298706e+02]
 [6.36045105e+02]
 [8.73231995e+02]
 [4.47892120e+02]
 [7.73516895e+03]
 [7.91493164e+02]
 [2.11136475e+02]
 [2.40449005e+02]
 [1.11060356e+02]
 [9.61276550e+02]
 [5.25753746e+01]
 [1.784788

3_Default_Xgboost rmse 3354.654603 trained in 95.49 seconds
4_Default_NeuralNetwork rmse 89483154.315097 trained in 59.69 seconds


/Users/gavinrolls/anaconda3/envs/urbsim/lib/python3.11/site-packages/supervised/utils/shap.py:38: UserWarning: Disable SHAP explanations because of number of columns > 500.


5_Default_RandomForest rmse 4216.065703 trained in 96.04 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 3354.654901 trained in 0.06 seconds
AutoML fit time: 335.76 seconds
AutoML best model: 3_Default_Xgboost


AutoML(results_path='automl_results/first_model/')